In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import openpyxl
import os
from time import sleep
import time
import re
import math
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller as AutoChrome

In [3]:
import chromedriver_autoinstaller as AutoChrome
from webdriver_manager.chrome import ChromeDriverManager

In [116]:
# chromedriver 열기
driver = webdriver.Chrome(ChromeDriverManager().install())

# 카카오맵 주소로 이동
driver.get('https://map.kakao.com/')  

driver.implicitly_wait(2) # 2초 기다리기

# 검색 창 
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') 
# 검색어 입력
search_area.send_keys('압구정동카페')  

# Enter로 검색
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER) 
driver.implicitly_wait(3) # 3초 기다리기

/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/2079174940.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/2079174940.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/2079174940.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)


In [110]:
urls = list()

def get_info():
    
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    cafe_lists = soup.select('ul.placelist>li.PlaceItem')
    
    for cafe in cafe_lists:
        
        #cafe_name = cafe.select('.head_item > .tit_name > .link_name')[0].get("title")
        url = cafe.select('.contact > .moreview')[0].get("href")
        
        #temp.append(cafe_name)
        urls.append(url)
        
       

In [117]:
#장소 더보기 누르기
more_place_button=driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]')
driver.execute_script("arguments[0].click();", more_place_button) 

In [118]:
page =1
page2 =0

for i in range(0,34):
    
    try:
        page2+=1
        print("**",page,"**")
        
        driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)
        get_info()
        
        if (page2)%5==0:
            driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            page2=0
            
        page+=1
    except:
        break
print("**크롤링완료**")
        
            


** 1 **


/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/1560840779.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)


** 2 **
** 3 **
** 4 **
** 5 **


/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/1560840779.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)


** 6 **
** 7 **
** 8 **
** 9 **
** 10 **
** 11 **
** 12 **
** 13 **
** 14 **
** 15 **
** 16 **
** 17 **
** 18 **
** 19 **
** 20 **
** 21 **
** 22 **
** 23 **
** 24 **
** 25 **
** 26 **
** 27 **
** 28 **
** 29 **
** 30 **
** 31 **
** 32 **
** 33 **
** 34 **
**크롤링완료**


In [119]:
urls_df = pd.DataFrame(urls)

In [120]:
urls_df.to_csv('압구정동_cafe_urls.csv')

In [125]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/mz/r1wtfk253597k9dz_sl4141r0000gn/T/ipykernel_76544/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [126]:
total_review=pd.DataFrame(columns=["store_name","review","star","id_count","id_star"]) #전체 리뷰 저장할 데이터 프레임. 

In [127]:
for url in urls:
    driver.get(url)
    print("접속 완료")
    driver.implicitly_wait(time_to_wait=2)
    
    try:
        review_count=driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g")
        if review_count.text=="(0)":
            print("후기 없음")
            continue
    except:#후기 미제공
        print("후기 미제공")
        continue
      
    try:
        more_info_button=driver.find_element(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > a') #더보기 버튼
        while True:
            if more_info_button.get_attribute("class")=='link_more link_unfold':
                break
            driver.execute_script("arguments[0].click();", more_info_button) 
        print("리뷰 내리기 완료")
        #오류: 못 나옴. 해결->css로 버튼에 접근. 
        if more_info_button.get_attribute("class")=='link_more':
            driver.execute_script("arguments[0].click();", more_info_button)
    except:
        print("내릴 스크롤이 없음. ")
        pass 
        
    

    
    try:
        while True :
            try:
                more_btn=driver.find_elements(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > a')[0].click()
                time.sleep(0.7)
            except:
                break
        print("더보기 모두 클릭 완료")
    except:
        pass

    store_soup = BeautifulSoup(driver.page_source, 'html.parser')
    store_name = store_soup.find("h2","tit_location").get_text()

    reviews = store_soup.find("div","evaluation_review")



    #별점
    star=reviews.find_all('span',{'class':'ico_star inner_star'})
    review_star=list()

    for i in star:
        real_star=int(i['style'].split(":")[1].split("%")[0])

        real_star/=20
        review_star.append(real_star)


    #리뷰 텍스트 
    review_text = list()

    texts = reviews.find_all('p','txt_comment')

    for txt in texts:
        txt = txt.get_text()
        review_text.append(txt)
    
    # id 당 리뷰개수, 별점평균
    id_count_star = list()

    counts = reviews.find_all('span', 'txt_desc')

    for i in counts:
        count = i.get_text()
        id_count_star.append(count)

    id_count = id_count_star[0::2]
    id_star = id_count_star[1::2]

    df = pd.DataFrame(columns=["store_name","review","star","id_count","id_star"])
    df["review"]=review_text
    df["star"]=review_star
    df["store_name"]=store_name
    df["id_count"]=id_count
    df["id_star"]=id_star
    
    total_review = total_review.append(df,ignore_index=True)
    print('한 가게 크롤링 완료')

    

접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료


더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료

후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 미제공
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
리뷰 내리기 

후기 없음
접속 완료
후기 없음
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 

접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
후기 없음
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
후기 없음
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
내릴 스크롤이 없음. 
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료
접속 완료
리뷰 내리기 완료
더보기 모두 클릭 완료
한 가게 크롤링 완료


In [128]:
total_review

,store_name,review,star,id_count,id_star
0,안녕인사동,더보기,3.0,53,4.6
1,안녕인사동,더보기,5.0,95,4.0
2,안녕인사동,인사동의 새로운 랜드마크가 되었으나 할 건 별로 없다. 근데 그건 쌈지길도 마찬가지...,4.0,5,4.0
3,안녕인사동,하리보 100주년 기념으로 놀러왔는데 같이. 온사람이 안해주고 불공평하게 이용하지맙...,1.0,2,3.0
4,안녕인사동,더보기,5.0,181,3.4
...,...,...,...,...,...
48137,설빙 가로수길점,더보기,1.0,1,1.0
48138,설빙 가로수길점,일점이 아까울 수도 있네요.더보기,1.0,3,3.7
48139,설빙 가로수길점,설빙이 어케 1점이지 뭐가 다르겠어하고 갔다가 왜 1점인지 알겠다 .. 빙수 잘못나...,1.0,42,3.7
48140,설빙 가로수길점,더보기,1.0,2,1.0


In [130]:
total_review.to_csv('종로1234동압구정동.csv')

In [105]:
seocho=pd.read_csv('종로1234동.csv')